# Credit Scoring


[Inspired by rorodata](https://github.com/rorodata/credit-scoring-demo)

In [1]:
import pandas as pd, numpy as np
import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.model_selection import GridSearchCV

import sklearn
#from sklearn_pandas import DataFrameMapper

import matplotlib.pyplot as plt
from IPython.display import HTML

In [2]:
df = pd.read_excel("data/mini-lending-club-data.xlsx")

In [3]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
2,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
3,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1
4,1072053,1288686,3000,3000,3000,36 months,18.64,109.43,E,E1,...,0.2,1.0,1.0,1.0,0,2.73575,20141201T000000,1,1,1


In [4]:
pd.set_option('display.max_colwidth', -1)
display(HTML(df.head().to_html()))
pd.reset_option('display.max_colwidth')

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,is_inc_v,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,not_compliant,status,inactive_loans,bad_loans,emp_length_num,grade_num,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000,Verified,20111201T000000,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077501,Borrower added on 12/22/11 > I need to upgrade my business technologies.<br>,credit_card,Computer,860xx,AZ,27.65,0.0,19850101T000000,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.07,5831.78,5000.0,861.07,0.00,0.0,0.0,20150101T000000,171.62,NaN,20150101T000000,0.0,NaN,1,0,Fully Paid,1,0,11,5,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252,Not Verified,20111201T000000,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077175,NaN,small_business,real estate business,606xx,IL,8.72,0.0,20011101T000000,2.0,NaN,NaN,2.0,0.0,2956,98.5,10.0,f,0.0,0.0,3003.65,3003.65,2400.0,603.65,0.00,0.0,0.0,20140601T000000,649.91,NaN,20150201T000000,0.0,NaN,1,0,Fully Paid,1,0,11,4,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
2,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200,Source Verified,20111201T000000,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1076863,"Borrower added on 12/21/11 > to pay for property tax (borrow from friend, need to pay back) & central A/C need to be replace. I'm very sorry to let my loan expired last time.<br>",other,personel,917xx,CA,20.00,0.0,19960201T000000,1.0,35.0,NaN,10.0,0.0,5598,21.0,37.0,f,0.0,0.0,12226.30,12226.30,10000.0,2209.33,16.97,0.0,0.0,20150101T000000,357.48,NaN,20150101T000000,0.0,NaN,1,0,Fully Paid,1,0,11,4,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
3,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,Veolia Transportaton,3 years,RENT,36000,Source Verified,20111201T000000,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1075269,NaN,wedding,My wedding loan I promise to pay back,852xx,AZ,11.20,0.0,20041101T000000,3.0,NaN,NaN,9.0,0.0,7963,28.3,12.0,f,0.0,0.0,5631.38,5631.38,5000.0,631.38,0.00,0.0,0.0,20150101T000000,161.03,NaN,20150201T000000,0.0,NaN,1,0,Fully Paid,1,0,4,6,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1
4,1072053,1288686,3000,3000,3000,36 months,18.64,109.43,E,E1,MKC Accounting,9 years,RENT,48000,Source Verified,20111201T000000,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1072053,Borrower added on 12/16/11 > Downpayment for a car.<br>,car,Car Downpayment,900xx,CA,5.35,0.0,20070101T000000,2.0,NaN,NaN,4.0,0.0,8221,87.5,4.0,f,0.0,0.0,3938.14,3938.14,3000.0,938.14,0.00,0.0,0.0,20150101T000000,111.34,NaN,20141201T000000,0.0,NaN,1,0,Fully Paid,1,0,10,2,0.2,1.0,1.0,1.0,0,2.73575,20141201T000000,1,1,1


In [5]:
features = ['grade',                     # grade of the loan (categorical)
            'sub_grade_num',             # sub-grade of the loan as a number from 0 to 1
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'payment_inc_ratio',         # ratio of the monthly payment to income
            'delinq_2yrs',               # number of delinquincies 
            'delinq_2yrs_zero',          # no delinquincies in last 2 years
            'inq_last_6mths',            # number of creditor inquiries in last 6 months
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'open_acc',                  # number of open credit accounts
            'pub_rec',                   # number of derogatory public records
            'pub_rec_zero',              # no derogatory public records
            'revol_util']                # percent of available credit being used

In [6]:
X = df[features]
y = df['bad_loans']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, 
                                                    stratify=y)

In [8]:
num = ['sub_grade_num', 'short_emp', 'emp_length_num', 'dti', 
       'payment_inc_ratio', 'delinq_2yrs', 'delinq_2yrs_zero',
       'inq_last_6mths', 'last_delinq_none', 'last_major_derog_none', 
       'open_acc', 'pub_rec', 'pub_rec_zero', 'revol_util']

cat = ['grade', 'home_ownership', 'purpose']

In [9]:
X_train.grade.value_counts()

B    3805
C    2960
A    2940
D    2231
E    1258
F     618
G     188
Name: grade, dtype: int64

In [10]:
X_train.home_ownership.value_counts()

RENT        6703
MORTGAGE    6200
OWN         1077
OTHER         20
Name: home_ownership, dtype: int64

In [11]:
X_train.purpose.value_counts()

debt_consolidation    7095
credit_card           1938
other                 1444
home_improvement       953
small_business         641
major_purchase         567
car                    389
wedding                256
medical                241
moving                 210
vacation               147
house                  119
Name: purpose, dtype: int64

In [12]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [13]:
le_grade = preprocessing.LabelEncoder()
le_own = preprocessing.LabelEncoder()
le_purpose = preprocessing.LabelEncoder()

enc_grade = OneHotEncoder(sparse=False)
enc_own = OneHotEncoder(sparse=False)
enc_purpose = OneHotEncoder(sparse=False)

grade = le_grade.fit_transform(X_train.grade)
home_ownership = le_own.fit_transform(X_train.home_ownership)
purpose = le_purpose.fit_transform(X_train.purpose)

grade = enc_grade.fit_transform(grade.reshape(-1,1))
home = enc_own.fit_transform(home_ownership.reshape(-1,1))
purpose = enc_purpose.fit_transform(purpose.reshape(-1,1))

In [14]:
X_train = pd.concat([X_train[num],
                     pd.DataFrame(np.hstack((grade, home, purpose)))], axis=1)

In [15]:
comb = pd.concat([X_train, y_train], axis=1)

In [16]:
comb = comb.dropna()

In [17]:
comb.shape

(9806, 38)

In [18]:
y_train = comb['bad_loans']

In [19]:
X_train = comb[X_train.columns]

In [20]:
test_grade = le_grade.transform(X_test.grade)
test_home_ownership = le_own.transform(X_test.home_ownership)
test_purpose = le_purpose.transform(X_test.purpose)

test_grade = enc_grade.fit_transform(test_grade.reshape(-1,1))
test_home = enc_own.fit_transform(test_home_ownership.reshape(-1,1))
test_purpose = enc_purpose.fit_transform(test_purpose.reshape(-1,1))


In [21]:
X_test = pd.concat([X_test[num],
                     pd.DataFrame(np.hstack((test_grade, test_home, test_purpose)))], axis=1)

In [22]:
test_comb = pd.concat([X_test, y_test], axis=1)

In [23]:
test_comb = test_comb.dropna()

In [24]:
test_comb.shape

(1745, 38)

In [25]:
y_test = test_comb['bad_loans']

In [26]:
X_test = test_comb[X_test.columns]

In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Model 1:

In [28]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
print(confusion_matrix(clf.predict(X_train), y_train))
print(accuracy_score(clf.predict(X_train), y_train))
print(confusion_matrix(clf.predict(X_test), y_test))
print(accuracy_score(clf.predict(X_test), y_test))

[[6914 2709]
 [  86   97]]
0.71497042627
[[1727    0]
 [  18    0]]
0.989684813754


# Model 2:

In [29]:
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)
print(confusion_matrix(clf.predict(X_train), y_train))
print(accuracy_score(clf.predict(X_train), y_train))
print(confusion_matrix(clf.predict(X_test), y_test))
print(accuracy_score(clf.predict(X_test), y_test))

[[6961 2613]
 [  39  193]]
0.729553334693
[[1724    0]
 [  21    0]]
0.987965616046


# Model 3:

In [30]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
print(confusion_matrix(clf.predict(X_train), y_train))
print(accuracy_score(clf.predict(X_train), y_train))
print(confusion_matrix(clf.predict(X_test), y_test))
print(accuracy_score(clf.predict(X_test), y_test))

[[7000    0]
 [   0 2806]]
1.0
[[1259    0]
 [ 486    0]]
0.721489971347


# Model 4:

In [31]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print(confusion_matrix(clf.predict(X_train), y_train))
print(accuracy_score(clf.predict(X_train), y_train))
print(confusion_matrix(clf.predict(X_test), y_test))
print(accuracy_score(clf.predict(X_test), y_test))

[[6996  224]
 [   4 2582]]
0.976748929227
[[1625    0]
 [ 120    0]]
0.931232091691


***